In [6]:
# Imports
from utils import read_json_file, write_json_file, read_json_file_or_empty_list
from utils import large_num_short_format
from dotenv import load_dotenv
from typing import  List, Dict
from utils import print_progress
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import  math

load_dotenv()

True

In [7]:
# Load coins and markets
from download_chart_data import ChartInterval
from download_chart_data import load_chart_by_interval

coins: List[Dict[str, any]] = read_json_file("data/coins.json")
markets: List[Dict[str, any]] = read_json_file("data/markets.json")
print("\nCoins: {}, Markets: {}".format(len(coins), len(markets)))

# Load daily charts and convert them to daily 
charts = load_chart_by_interval(ChartInterval.FULL)
print("\nCharts with price data count: {}".format(len(charts)))


Coins: 10431, Markets: 10431
 Loading chart_full idx: 10424 / 10425, 100.0%       
Charts with price data count: 9887


In [8]:
# Find chart pumps
from download_chart_data import chart_data_to_df
from utils import year_month_str, year_month_day_str

def is_pump(
    price: float, prev_price: float, pump_ratio: float, volume: float, 
    min_volume: float
) -> bool:
    if math.isnan(prev_price) or math.isnan(price):
        return False
    return price >= prev_price * pump_ratio and volume > min_volume


def pumps_info(
    chart_data: List[Dict[str, any]], pump_ratio: float, min_volume: float
) -> pd.DataFrame:
    df = chart_data_to_df(chart_data)
    df['Ym'] = list(map(lambda x: year_month_str(x), df.index))
    # shift price and add pump bool
    df.insert(1, 'prev_price', df['price'].shift(1))
    df['pump'] = df.apply(
        lambda x: is_pump(x['price'], x['prev_price'], pump_ratio, x['volume'], min_volume),
        axis=1
    )
    # select pump rows
    return df.loc[df['pump'] == True]


pump_ratio = 2.0
min_volume = 500000
chart_pumps_df: Dict[str, pd.DataFrame] = dict()

# for each chart
for idx, key in enumerate(charts):
    chart = charts[key]
    print_progress("Searching for pumps", idx, len(charts), True)
    chart_pumps_df[key] = pumps_info(chart, pump_ratio, min_volume)

print("")        
print("Coins that pumped", len(chart_pumps_df.keys()))

 Searching for pumps idx: 9886 / 9887, 100.0%       
Coins that pumped 9887


In [9]:
# Filter out only ETH  pumps

def get_address_and_decimals(coin_id: str) -> (str, int):
    try: 
        coin = read_json_file('data/coin/' + coin_id + '.json')
    except:
        return None, None
    detail_platforms = coin.get('detail_platforms')
    if detail_platforms is None:
        return None, None
    ethereum = detail_platforms.get('ethereum')
    if ethereum is None:
        return None, None
    return ethereum.get('contract_address'), ethereum.get('decimal_place')


eth_df_candles_pumps = dict()

for key, df in chart_pumps_df.items():
    address, decimals = get_address_and_decimals(key)
    if address is None or decimals is None or len(df) == 0:
        continue    
    eth_df_candles_pumps[key] = df
        
print(sorted(eth_df_candles_pumps.keys()))
print("Eth coins pumps", len(eth_df_candles_pumps.keys()))

tmp_id = 'anchor-protocol'
tmp_df = eth_df_candles_pumps[tmp_id]
print(tmp_id, tmp_df.shape)
for i in range(len(tmp_df.index)):
    print(year_month_day_str(tmp_df.index[i]))

['0xcoco', '12ships', '1million-nfts', '1sol', '2crazynft', '88mph', 'aardvark', 'acent', 'acoconut', 'acreage-coin', 'acute-angle-cloud', 'adappter-token', 'adex', 'adreward', 'advanced-united-continent', 'aegis', 'aelf', 'aeron', 'agoras-currency-of-tau', 'agrello', 'agrinode', 'airbloc-protocol', 'airswap', 'aiwork', 'akita-inu', 'akropolis-delphi', 'alchemy-pay', 'alethea-artificial-liquid-intelligence-token', 'all-coins-yield-capital', 'all-in', 'ally', 'alpaca', 'alpha-quark-token', 'alphr', 'aluna', 'amber-phantom-butterfly', 'american-shiba', 'amond', 'anchor-protocol', 'answer-governance', 'antis-inu', 'anyswap', 'apes-go-bananas', 'apeswap-finance', 'apex-token-2', 'apm-coin', 'apollo-crypto', 'apy-finance', 'arbismart-token', 'arcblock', 'archangel-token', 'archimedes', 'arcona', 'arcs', 'armor', 'asd', 'ash', 'askobar-network', 'assangedao', 'assemble-protocol', 'astroelon', 'attila', 'auctus', 'aurix', 'aurora', 'aurora-dao', 'autobahn-network', 'automata', 'autonio', 'ave

In [10]:
from web3_utils import format_decimals, get_block_near
from utils import read_str_file, year_month_day_str
from web3 import Web3, AsyncWeb3
import web3
import datetime

provider = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))

# Create two data frames for analyses
# Ymd from to val timestamp, block, tx_hash

def block_num_for_timestamp(timestamp: int, provider: web3.Web3) -> int:
    return get_block_near(timestamp, provider)
    

def log_to_list(log: any, decimals: int, provider: web3.Web3) -> List[any]:
    val_int = log.args['value']
    timestamp = provider.eth.get_block(log['blockNumber'])['timestamp']
    return [
        provider.to_hex(log['transactionHash']), # tx_hash
        log['transactionIndex'],                 # tx_idx       
        int(log['blockNumber']),                 # block_num
        log['blockHash'],                        # block_hash
        timestamp,                               # timestamp
        log.args['from'],                        # frm
        log.args['to'],                          # to
        val_int,                                 # val_int
        format_decimals(val_int, str(decimals)), # val_dec
        year_month_day_str(timestamp),           # Ymd
    ]


# tx_hash block_num timestamp frm to val_int val_dec Ymd
def get_logs_df(
    coin_id: str, 
    pump_timestamp: int,
    days: int,
    provider: web3.Web3
) -> pd.DataFrame:
    # if not ETH return None
    address, decimals = get_address_and_decimals(coin_id)
    if address is None or decimals is None:
        return None
    pump_block_num = block_num_for_timestamp(pump_timestamp, provider)
    st_block = pump_block_num - days * 24 * 60 * 5
    
    # crawl all transfer logs
    cols = ['tx_hash', 'tx_idx', 'block_num', 'block_hash', 'timestamp', 'frm', 'to', 'val_int', 'val_dec', 'Ymd']
    df = pd.DataFrame(columns=cols)
    abi = read_str_file('IERC20.json')
    address, decimals = get_address_and_decimals(coin_id)
    address = Web3.to_checksum_address(address)
    contract = provider.eth.contract(address=address,abi=abi)
    batch_size = 1000
    to = st_block + batch_size
    prog_total = pump_block_num - st_block
    while to < pump_block_num + (3 * 24 * 60 * 5):
        frm = to - batch_size
        msg = "{} {} {}".format(coin_id, frm, to)
        print_progress(msg, to - st_block, prog_total, True)
        batch_logs = contract.events.Transfer().get_logs(fromBlock=frm, toBlock=to)
        for log in batch_logs:
            df.loc[len(df.index)] = log_to_list(log, decimals, provider)
        to += batch_size
    return df


coin_id = "anchor-protocol"
# 0x7da2641000cbb407c329310c461b2cb9c70c3046
df = eth_df_candles_pumps[coin_id]
print(df.shape)
timestamp = df.index[1]
print(year_month_day_str(timestamp))
df_logs = get_logs_df(coin_id, timestamp, 90, provider)
# tx_hash block_num timestamp frm to val_int val_dec Ymd 

(2, 6)
2022-05-24
 anchor-protocol 14852563 14853563 idx: 669000 / 648000, 103.2%       

In [11]:
def flatten_logs_to_tx(coin_id: str, df: pd.DataFrame, provider: web3.Web3) -> pd.DataFrame:
    df = df.copy()
    df.drop_duplicates('tx_hash', inplace = True)
    print(df.shape)
    df.insert(df.shape[1], 'method', np.arange(0, df.shape[0]))
    for i in range(len(df.index)):
        idx = df.index[i]
        tx_hash = df.loc[idx, 'tx_hash']
        tx_idx = df.loc[idx, 'tx_idx']
        block_num = df.loc[idx, 'block_num']
        print_progress('Fetching {}'.format(tx_hash), i, df.shape[0], True)
        tx = provider.eth.get_transaction_by_block(int(block_num), int(tx_idx))
        input = provider.to_hex(tx.input) 
        method = input[:10] 
        df.loc[idx, 'frm'] = tx['from']
        if method == '0xa9059cbb':
            df.loc[idx, 'method'] = 'transfer'
            df.loc[idx, 'to'] = "0x" + input[34:74] 
        else:
            df.loc[idx, 'method'] = 'unknown'
    
    return df


df_tx = flatten_logs_to_tx(coin_id, df_logs, provider)

(2097, 10)
 Fetching 0x3d093c3c20cff497aa9f83220cfa1f440195765878689584a03e7ac34d8714d4 idx: 2096 / 2097, 100.0%       

In [13]:
from math import ceil


# Ymd tx_cnt, tx_val_total, eoa_tx_cnt, eoa_val_total
def grouped_by_day(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    grouped = df.groupby('Ymd')
    res_df = pd.DataFrame(columns=['Ymd', 'txs_cnt', 'txs_val', 'eoa_txs_cnt', 'eoa_txs_val'])
    print("groups", len(grouped))
    for key, group in grouped:
        eoa_txs = group[group['method'] == 'transfer']
        eoa_txs_cnt = len(eoa_txs)
        eoa_txs_val = ceil(eoa_txs['val_dec'].sum())
        row = [key, len(group), ceil(group['val_dec'].sum()), eoa_txs_cnt, eoa_txs_val]
        res_df.loc[len(res_df.index)] = row
    
    res_df['tx_cnt_pct'] =  res_df['eoa_txs_cnt'] / res_df['txs_cnt']
    res_df['tx_val_pct'] =  res_df['eoa_txs_val'] / res_df['txs_val']
    res_df['high_swap'] =  res_df['tx_val_pct'] < 0.5 # res_df['eoa_txs_val'] / res_df['txs_val']
    return res_df


pd.set_option('display.width', 256)
pd.set_option('display.max_colwidth', 24)

# Ymd tx_cnt, tx_val_total, eoa_tx_cnt, eoa_val_total
res_df = grouped_by_day(df_tx)
print(res_df.to_string())

pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')

groups 102
            Ymd  txs_cnt  txs_val  eoa_txs_cnt  eoa_txs_val  tx_cnt_pct  tx_val_pct  high_swap
0    2022-02-11        3     8265            2         6190    0.666667    0.748941      False
1    2022-02-12       13    64721           13        64721    1.000000    1.000000      False
2    2022-02-13       13    27861           12        27631    0.923077    0.991745      False
3    2022-02-14       13   210341           10       108188    0.769231    0.514346      False
4    2022-02-15       21    73054           21        73054    1.000000    1.000000      False
5    2022-02-16       11    32579            8        27710    0.727273    0.850548      False
6    2022-02-17        5    82242            4        52384    0.800000    0.636949      False
7    2022-02-18        3    64641            2        34794    0.666667    0.538265      False
8    2022-02-19       22    80964           20        80073    0.909091    0.988995      False
9    2022-02-20        6    44016      